# Merge collected data

- ``IEDB.csv``, ``VDJdb_pair.csv``, ``McPAS-TCR.csv``

- peptide-HLA-TCR

  - TCR with alpha/beta chain, v/j-gene, cdr3 

- human(HLA)

In [2]:
import os
import pandas as pd
from utils import save_csv, plot_non_empty_percentage

to_your_path = '/data/lujd/'
datapath = to_your_path + "TCRdata/collect/"

### ¶ read collected data

In [50]:
iedb_fn = "IEDB.csv"
iedb_df = pd.read_csv(datapath+iedb_fn,  sep=",", low_memory=False)
vdjdb_fn = "VDJdb_pair.csv"
vdjdb_df = pd.read_csv(datapath+vdjdb_fn,  sep=",", low_memory=False)
mcpas_fn = "McPAS-TCR.csv"
mcpas_df = pd.read_csv(datapath+mcpas_fn,  sep=",", low_memory=False)
print(len(iedb_df), len(vdjdb_df), len(mcpas_df))

# human
vdjdb_df = vdjdb_df[vdjdb_df['species'] == 'HomoSapiens']
# vdjdb_df = vdjdb_df[vdjdb_df['vdjdb.score'] >= 1]
mcpas_df = mcpas_df[mcpas_df['Species'] == 'Human']
print(len(iedb_df), len(vdjdb_df), len(mcpas_df))

205670 57551 15566
205670 53337 12737


In [51]:
vdjdb_df['vdjdb.score'].value_counts()

vdjdb.score
0    47562
1     3722
2     1383
3      670
Name: count, dtype: int64

### ¶ alignment

- IEDB/McPAS-TCR: Get MHC class

In [52]:
for mhc in iedb_df["mhc.allele"].unique()[7:17]:
    print(mhc.split(', '))

['HLA-A*02:01', 'HLA-A*02:01 K66A', 'E63Q mutant']
['HLA-A*02:01']
['HLA-A2']
['HLA-B*27:05']
['HLA-B*35:01']
['HLA-DRB1*04:01']
['HLA-DRB1*01:01']
['HLA-B*44:02', 'HLA-B*44:03', 'HLA-B*44:05']
['HLA-A*02:01 K66A', 'E63Q mutant']
['HLA-A*02:01 K66A mutant']


In [53]:
def split_mhc_allele(df):
    # split the lines based on the delimiter ";" and expand them into separate columns
    split_df = df['mhc.allele'].str.split(', ', expand=True)

    # Stack the split lines into multiple rows, reset index at level 1 and drop none element
    stacked_df = split_df.stack().reset_index(level=1, drop=True).to_frame(name='mhc.allele')

    # Concatenate it with the original DataFrame and reset the index for the final DataFrame
    # result_df = pd.concat([df.drop(['Amino Acids'], axis=1), stacked_df], axis=1).reset_index(drop=True)
    result_df = df.drop(['mhc.allele'], axis=1).join(stacked_df).reset_index(drop=True)

    return result_df

iedb_df = split_mhc_allele(iedb_df)
iedb_df.drop_duplicates(inplace=True, ignore_index=True)
print(len(iedb_df))

205871


In [54]:
iedb_df['mhc.allele'].nunique()

150

In [55]:
def mhc_allele2class(data):
    if str(data) != 'nan':
        return 'MHCII' if ('D' in str(data)) or ('II' in str(data)) else 'MHCI'
    else:
        return '(UNK)'

def check_mhc_allele(data):
    if (str(data) == 'nan') or ('class' in str(data)):
        return '(UNK)'
    else:
        return data

for mhc in iedb_df['mhc.allele'].unique():
    print(mhc_allele2class(mhc), mhc)
for mhc in iedb_df['mhc.allele'].unique():
    print(check_mhc_allele(mhc), mhc)

MHCI HLA-E*01:01
MHCI HLA-E*01:03
MHCI HLA-B8
MHCI HLA-B*44:05
MHCI HLA-B*08:01
MHCI HLA-B*44:02
(UNK) nan
MHCI HLA-A*02:01
MHCI HLA-A*02:01 K66A
MHCI E63Q mutant
MHCI HLA-A2
MHCI HLA-B*27:05
MHCI HLA-B*35:01
MHCII HLA-DRB1*04:01
MHCII HLA-DRB1*01:01
MHCI HLA-B*44:03
MHCI HLA-A*02:01 K66A mutant
MHCI HLA-A*02:01 A150P mutant
MHCI HLA-B*35:08
MHCI HLA-B*35:08 Q65A
MHCI T69A
MHCI Q155A mutant
MHCII HLA-DRA*01:01/DRB1*15:01
MHCII HLA-DRA*01:01/DRB1*04:01
MHCII HLA-DQ1
MHCII HLA-DQA1*01:02/DQB1*05:02
MHCII HLA-DRB3*03:01
MHCII HLA-DR
MHCII HLA-DRB1*04:04
MHCII HLA-DRB1*08:03
MHCII HLA-DRB1*09:01
MHCII HLA-DRB1*15:01
MHCII HLA-DRB1*15:03
MHCII HLA-DRB1*13:02
MHCI HLA-B*57:01
MHCI HLA-B*57:03
MHCI HLA-A*24:02
MHCII HLA-DQA1*05:01/DQB1*02:01
MHCI HLA-B*51:01
MHCII HLA-DPB1*02:01
MHCII HLA-DQA1*03:01/DQB1*03:02
MHCI HLA-A*01:01
MHCII HLA class II
MHCII HLA-DQA1*05:01/DQB1*03:02
MHCII HLA-DR2
MHCII H2 class II
MHCII HLA-DRB3*01:01
MHCI HLA-A11
MHCI HLA-B*07:02
MHCI HLA-B7
MHCI HLA-A*11:01
MHCI 

In [56]:
iedb_df['mhc.class'] = iedb_df['mhc.allele'].apply(mhc_allele2class)
iedb_df['mhc.allele'] = iedb_df['mhc.allele'].apply(check_mhc_allele)

vdjdb_df = vdjdb_df.rename(columns={'mhc.a':'mhc.allele'})

mcpas_df['mhc.class'] = mcpas_df['mhc.allele'].apply(mhc_allele2class)
mcpas_df['mhc.allele'] = mcpas_df['mhc.allele'].apply(check_mhc_allele)

### ¶ add dataset.tag

In [59]:
iedb_df['dataset.tag'] = 'iedb'
vdjdb_df['dataset.tag'] = 'vdjdb'
mcpas_df['dataset.tag'] = 'mcpas'

### ¶ replace empty-value with (UNK)

In [60]:
iedb_df = iedb_df.fillna(value='(UNK)')
vdjdb_df = vdjdb_df.fillna(value='(UNK)')
mcpas_df = mcpas_df.fillna(value='(UNK)')

vdjdb_df['alpha.cdr1'] = '(UNK)'
vdjdb_df['alpha.cdr2'] = '(UNK)'
vdjdb_df['alpha.vseq'] = '(UNK)'
vdjdb_df['beta.cdr1'] = '(UNK)'
vdjdb_df['beta.cdr2'] = '(UNK)'
vdjdb_df['beta.vseq'] = '(UNK)'

mcpas_df['alpha.cdr1'] = '(UNK)'
mcpas_df['alpha.cdr2'] = '(UNK)'
mcpas_df['alpha.vseq'] = '(UNK)'
mcpas_df['beta.cdr1'] = '(UNK)'
mcpas_df['beta.cdr2'] = '(UNK)'
mcpas_df['beta.vseq'] = '(UNK)'

### ¶ merge peptide-HLA-TCR data

- TCR with alpha/beta chain, v/j-gene, cdr3 

In [63]:
main_cols = ['antigen.epitope','mhc.allele','mhc.class',
             'alpha.v','alpha.j','alpha.cdr1','alpha.cdr2','alpha.cdr3','alpha.vseq',
             'beta.v','beta.d','beta.j','beta.cdr1','beta.cdr2','beta.cdr3','beta.vseq',
             'dataset.tag']

iedb_df_ab_vdj = iedb_df[main_cols]
vdjdb_df_ab_vdj = vdjdb_df[main_cols]
mcpas_df_ab_vdj = mcpas_df[main_cols]

df_ab_vdj = pd.concat([iedb_df_ab_vdj, vdjdb_df_ab_vdj, mcpas_df_ab_vdj]
                        ).drop_duplicates(subset=['antigen.epitope','mhc.allele','mhc.class',
                                                  'alpha.v','alpha.j','alpha.cdr1','alpha.cdr2','alpha.cdr3','alpha.vseq',
                                                  'beta.v','beta.d','beta.j','beta.cdr1','beta.cdr2','beta.cdr3','beta.vseq'], keep='first'
                        ).reset_index(drop=True)
print(len(df_ab_vdj))       # 193692 if only use drop_duplicates()

193800


### ¶ Save data to ``TCRdata/collect/``

In [64]:
# save
print(f"number of brief data: {len(df_ab_vdj)}")

df_ab_vdj = df_ab_vdj.drop_duplicates(ignore_index=True)
print(f"number of duplicated brief data: {len(df_ab_vdj)}")

savepath = to_your_path + "TCRdata/collect/"
savename = "main_dataset_tcr_phla.csv"
save_csv(df_ab_vdj, savepath, savename)

number of brief data: 193800
number of duplicated brief data: 193800
File /data/lujd/TCRdata/collect/main_dataset_tcr_phla.csv has been successfully saved.
